<a href="https://colab.research.google.com/github/JoaoPauloSousaCoelho/Joao-Paulo/blob/master/DQN_TSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: build a bayesian optimization algorithm. Considering to test different neural  architecture. Please, test in differents instances and calculate the gap (ratio between best distance and the optimum distance of the instance). Please save the results (generating a csv file) (distances and gaps found, values of hiperparameters and name of instances)

# --- Data Collection and CSV Saving ---

# Store results from Bayesian Optimization runs
results_list = []

@use_named_args(space)
def objective(**params):
    """
    Objective function for Bayesian Optimization.
    Evaluates performance across different TSP instances and stores results.
    """
    print("\nEvaluating hyperparameters:")
    print(params)

    instance_scores = {}
    for instance_name, cities_coords in TSP_INSTANCES.items():
        print(f"  Evaluating on instance: {instance_name}")
        env = TSPEnvironment(cities_coords)
        state_input_shape = env.num_cities + env.num_cities + (env.num_cities * 2)
        action_size = env.num_cities
        optimal_distance = 22 if instance_name == "eil7" else None # Define optimal distances for known instances

        try:
            agent = DQNAgent(
                state_size=state_input_shape,
                action_size=action_size,
                learning_rate=params['learning_rate'],
                gamma=params['gamma'],
                epsilon_start=1.0,
                epsilon_end=params['epsilon_end'],
                epsilon_decay_rate=params['epsilon_decay_rate'],
                replay_buffer_size=params['replay_buffer_size'],
                batch_size=params['batch_size'],
                target_update_freq=params['target_update_freq'],
                n_hidden_layers=params['n_hidden_layers'],
                n_neurons=params['n_neurons']
            )

            instance_avg_tour_length = train_dqn_for_optimization(env, agent, num_episodes=200)

            agent.cleanup()
            del agent
            gc.collect()
            tf.keras.backend.clear_session()
            gc.collect()

            instance_scores[instance_name] = instance_avg_tour_length
            print(f"    Instance {instance_name} average tour length: {instance_avg_tour_length:.2f}")

        except Exception as e:
            print(f"    Training failed for instance {instance_name} with params {params}: {e}")
            instance_scores[instance_name] = 10000 # Penalty for failure

    # Store results for this set of hyperparameters
    result_entry = params.copy()
    result_entry['avg_score_across_instances'] = np.mean(list(instance_scores.values()))

    # Calculate and store gap for each instance (if optimal is known)
    for instance_name, avg_dist in instance_scores.items():
        result_entry[f'{instance_name}_distance'] = avg_dist
        if instance_name == "eil7" and 22 is not None: # Check if optimal is known for this instance
             result_entry[f'{instance_name}_gap'] = (avg_dist - 22) / 22 if 22 > 0 else float('inf')
        else:
             result_entry[f'{instance_name}_gap'] = None # Gap is None if optimal is unknown

    results_list.append(result_entry)

    return result_entry['avg_score_across_instances']

# --- Run Bayesian Optimization and Collect Results ---

print("Starting Bayesian Optimization with multi-instance evaluation and data collection...")

res_gp = gp_minimize(
    objective,
    space,
    n_calls=100,
    n_initial_points=20,
    random_state=42,
    verbose=True,
    n_jobs=1
)

print("\nBayesian Optimization finished.")
print(f"Best score (minimum average tour length across instances): {res_gp.fun:.2f}")
print("Best hyperparameters found:")
best_params = dict(zip([param.name for param in space], res_gp.x))
print(best_params)

# --- Save Results to CSV ---

print("\nSaving results to CSV...")

# Convert the list of results (dictionaries) into a pandas DataFrame
results_df = pd.DataFrame(results_list)

# Define the filename
csv_filename = "bayesian_optimization_results.csv"

# Save the DataFrame to a CSV file
results_df.to_csv(csv_filename, index=False)

print(f"Results saved to {csv_filename}")

# Optional: Display the first few rows of the results DataFrame
print("\nResults DataFrame Head:")
print(results_df.head())

Starting Bayesian Optimization with multi-instance evaluation and data collection...
Iteration No: 1 started. Evaluating function at random point.

Evaluating hyperparameters:
{'learning_rate': 0.002452612631133679, 'gamma': 0.8365035231833666, 'epsilon_end': 0.15814129005182623, 'epsilon_decay_rate': 0.9979245657739378, 'replay_buffer_size': np.int64(5012), 'batch_size': np.int64(83), 'target_update_freq': np.int64(165), 'n_hidden_layers': np.int64(2), 'n_neurons': np.int64(91)}
  Evaluating on instance: eil7
    Instance eil7 average tour length: 213.00
  Evaluating on instance: berlin52
    Instance berlin52 average tour length: 33284.00
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 66.8045
Function value obtained: 16748.5000
Current minimum: 16748.5000
Iteration No: 2 started. Evaluating function at random point.

Evaluating hyperparameters:
{'learning_rate': 0.0008967376801947966, 'gamma': 0.811225904226393, 'epsilon_end': 0.14717976673069674, 'epsilon_decay_